<a href="https://colab.research.google.com/github/juanpi19/medium-articles/blob/main/statistical-power/CUPED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uuid

import statsmodels.formula.api as smf
import scipy.stats as stats

# Data Generating Process

In [ ]:
df = pd.DataFrame()

# Sample Size
n = 1000
true_effect = 500

# User IDs
user_ids = [str(uuid.uuid4()) for _ in range(n)]

# Baseline Revenue: will be drawn from a normal distribution with 2000 mean, 300 variance. This refers to $$
baseline_revenue = np.random.normal(loc=2000, scale=300, size=n)

# noise
var = 10

# Dataframe
df = pd.DataFrame()
df['user_id'] = user_ids
df.loc[:499, 'treatment'] = 1 # Split users 50/50 to receive the treatment
df.loc[499:, 'treatment'] = 0

df['baseline_revenue'] = baseline_revenue
df['experiment_revenue'] = np.where(df['treatment'] == 0,
                                  baseline_revenue + np.random.normal(0,var,n),
                                  baseline_revenue + np.random.normal(loc=true_effect, scale=200, size=n) + np.random.normal(0,var,n))

In [ ]:
df.head()

,user_id,treatment,baseline_revenue,experiment_revenue
0,c943157b-7b46-4af2-a117-23a8b8a81fe6,1.0,2445.857936,2833.034084
1,580d98c2-05b2-4c88-9d35-d34d5dde6df7,1.0,2100.192045,2572.209342
2,4410935a-b569-4df5-8c13-89ff6b0e0230,1.0,1771.354559,2612.638197
3,151fe30f-ff9a-4b61-b055-ec262e4c8f78,1.0,2008.257007,2618.281138
4,c976c00e-d228-407c-918e-c37da17e492d,1.0,1882.083583,2499.545542


# T-test

In [ ]:
control = df[df['treatment'] == 0]['experiment_revenue']
treatment = df[df['treatment'] == 1]['experiment_revenue']

t_stat, p_value = stats.ttest_ind(treatment, control, equal_var=True)
effect_size = treatment.mean() - control.mean()

print("Effect size: ", effect_size.round(4))
print(f"T-statistic: ", t_stat.round(3))
print(f"P-value: ", p_value.round(3))

Effect size:  516.2068
T-statistic:  24.257
P-value:  0.0


# Linear Regression wo pre-experiment data

In [ ]:
model = smf.ols('experiment_revenue ~ treatment', data=df).fit()
print(model.summary().tables[1])

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1991.1748     15.033    132.456      0.000    1961.676    2020.674
treatment    516.2068     21.281     24.257      0.000     474.447     557.967


# Linear Regression with pre-experiment data

In [ ]:
model = smf.ols('experiment_revenue ~ treatment + baseline_revenue', data=df).fit()
print(model.summary().tables[1])

                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           32.0271     29.505      1.085      0.278     -25.872      89.926
treatment          506.8181      8.969     56.505      0.000     489.217     524.419
baseline_revenue     0.9840      0.014     67.986      0.000       0.956       1.012


In [ ]:
# Treatment Mean (in deltoid) = raw_mean_test + beta_test(pre_mean_pooled - pre_mean_test)

beta_test = model.params['baseline_revenue']
final_test_mean = np.mean(df[df['treatment']==1]['experiment_revenue']) + beta_test*(np.mean(df['baseline_revenue']) - np.mean(df[df['treatment']==1]['baseline_revenue']) )
final_control_mean = np.mean(df[df['treatment']==0]['experiment_revenue']) + beta_test*(np.mean(df['baseline_revenue']) - np.mean(df[df['treatment']==0]['baseline_revenue']) )
final_test_mean - final_control_mean

506.81806335645274